In [42]:
import requests
import json

url = "http://localhost:11434/api/generate"
payload = {
    "model": "llama3.2:3b",
    "prompt": "너는 반드시 한국어로만 대답해야 한다. 대한민국의 수도는 어디야?"
}


with requests.post(url, json=payload, stream=True) as r:
    for line in r.iter_lines():
        if line:
            data = json.loads(line.decode("utf-8"))
            if "response" in data:
                print(data["response"], end="", flush=True)
            if data.get("done", False):
                break


한국의 수도인 서울입니다.

In [43]:
# 경로 설정
PDF_FILE_PATH = '/Users/yoodongseok/Desktop/rag_project/data/2024 관세행정 민원상담 사례집.pdf'

In [44]:
# 텍스트 추출
import os
import pdfplumber

all_text = ""
if PDF_FILE_PATH and os.path.exists(PDF_FILE_PATH): # 파일 존재 여부 확인
    with pdfplumber.open(PDF_FILE_PATH) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text: 
                all_text += f"--- 페이지 {i+1} ---\n{text}\n\n"
    
    print("--- 추출된 텍스트 미리보기 (상위 500자) ---")
    print(all_text[:500])
else:
    print("파일 경로가 올바르지 않거나 파일이 존재하지 않습니다. 경로를 다시 확인해 주세요.")

--- 추출된 텍스트 미리보기 (상위 500자) ---
--- 페이지 1 ---
발 간 등 록 번 호
11-1220000-000430-10
2024
관세행정 민원상담 사례집

--- 페이지 2 ---
일러두기
이 책은 그동안의 관세행정 민원상담 사례를 모아 민원인에게
도움을 주고자 최신법령 등을 반영한 종합민원 사례집으로, 관련
법령 개정 및 민원의 구체적인 사항에 따라 이 책의 내용이 적용될
수 없을 수 있습니다.
따라서, 이 책의 내용을 쟁송, 유권해석 등의 근거자료로 사용하는
것은 적절하지 아니하며, 전반적인 관세행정에 관한 참고자료로
활용해 주시고, 관련 법령 등을 확인하여 주시기 바랍니다.

--- 페이지 3 ---
발 간 사
관세청은, 관세행정 전 분야의 끊임없는
혁신을 통해 대한민국이 글로벌 중추국가로
도약할 수 있도록 앞장서 나가겠습니다.
최근 관세행정은 해외직구 등 전자상거래 관련 개인 민원이 2023년 전체 민원
총 240,103건에서 54%를 차지할 정도로 특송화물‧우편물‧여행자 분야의
중심 고객이 기업에서 개인으로 


In [45]:
# 테이블 추출
import tabula
import pandas as pd
import os

if PDF_FILE_PATH and os.path.exists(PDF_FILE_PATH): # 파일 존재 여부 확인
    print("\n[Tabula-py] 표 데이터 추출 시작...")
    
    dfs_list = tabula.read_pdf(
        PDF_FILE_PATH, 
        pages='all',
        multiple_tables=True,
        stream=True, 
        encoding='utf-8'
    )

    if dfs_list:
        print(f"총 {len(dfs_list)}개의 표(DataFrame)를 추출했습니다.")
        for i, df in enumerate(dfs_list):
            print(f"\n--- 추출된 표 {i+1} (DataFrame.head()) ---")
            print(df.head())
    else:
        print("Tabula-py가 PDF 파일에서 표를 찾지 못했습니다.")
else:
    print("파일 경로가 올바르지 않거나 파일이 존재하지 않습니다. 경로를 다시 확인해 주세요.")

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'



[Tabula-py] 표 데이터 추출 시작...


Got stderr: 10월 04, 2025 12:19:18 오후 org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
10월 04, 2025 12:19:18 오후 org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode



총 28개의 표(DataFrame)를 추출했습니다.

--- 추출된 표 1 (DataFrame.head()) ---
  Unnamed: 0   품목     물품가격        면세수량           비고
0        NaN   주류      NaN  1병 (1L 이하)  주세, 교육세는 과세
1        NaN   궐련      NaN       200개비          NaN
2        NaN  엽궐련      NaN        50개비          NaN
3        NaN  NaN  미화 150불         NaN       개별소비세,
4        NaN  NaN      NaN  니코틴용액 20ml          NaN

--- 추출된 표 2 (DataFrame.head()) ---
  과세가격결정방법 사전심사신청서【일반용】
0          (처리기간 : 1개월)
1     신  1 상 호  2 대 표 자
2                     청
3       인  3 주 소 (전화번호)
4            4 수입 예정 물품

--- 추출된 표 3 (DataFrame.head()) ---
  Unnamed: 0     과세가격 결정방법 사전심사
0        NaN        신청서【특수관계자용】
1        NaN        (처리기간 : 1년)
2          신  1 법인명(상호) 2 대 표 자
3          청                NaN
4          인       3 주 소 (전화번호)

--- 추출된 표 4 (DataFrame.head()) ---
    품목(예시) 면세통관범위 품목(예시).1 면세통관범위.1
0      참기름    5kg        잣      1kg
1      참 깨    5kg      쇠고기     10kg
2        꿀    5kg     돼지고기     10KG
3  고사리, 더덕    5kg      그 외  품목당 5kg

In [51]:
import pdfplumber
import pandas as pd

file_path = PDF_FILE_PATH
qa_pairs = []

with pdfplumber.open(file_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        tables = page.extract_tables()

        lines = text.split("\n") if text else []
        key = None
        value_lines = []

        for line in lines:
            if "?" in line:  # 질문
                if key and (value_lines or tables):
                    # 표를 문자열로 합쳐서 추가
                    table_text = ""
                    if tables:
                        for t in tables:
                            for row in t:
                                table_text += " | ".join([c if c else "" for c in row]) + "\n"

                    qa_pairs.append({
                        "question": key,
                        "answer": " ".join(value_lines).strip() + "\n" + table_text.strip()
                    })
                    value_lines = []
                key = line.strip()
            elif key:
                value_lines.append(line.strip())

        if key and (value_lines or tables):
            table_text = ""
            if tables:
                for t in tables:
                    for row in t:
                        table_text += " | ".join([c if c else "" for c in row]) + "\n"

            qa_pairs.append({
                "question": key,
                "answer": " ".join(value_lines).strip() + "\n" + table_text.strip()
            })

# DataFrame → Parquet 저장
df = pd.DataFrame(qa_pairs)
df.to_parquet("qa_dataset.parquet", engine="fastparquet", index=False)
print("✅ Parquet 저장 완료")
print(df.head())


✅ Parquet 저장 완료
                                            question  \
0  10. 세관장확인물품으로 지정된 유아용 섬유제품이란?·················...   
1  20. 전기용품 수입시 어떤 절차를 거쳐야 하나요?··················...   
2  24. 환불받은 해외직구물품 타인 명의로 다시 수입 가능한지?············...   
3  26. 수입 신고는 어느 세관에 하여야 하나?·····················...   
4  28. 개인자격으로 영리목적 수입 가능한지?······················...   

                                              answer  
0  11. 주방용 쓰레기 처리기(Disposers) 수입요건···············...  
1  21. 술, 담배의 면세기준과 기준 초과시 수입절차··················...  
2  25. 수입물품에 대한 세액계산 방법··························...  
3                                관세행정 민원상담 사례집 | i\n  
4  29. B/L 분할 수입신고 요건 및 B/L 합병 후 수입신고 요건·········...  


In [79]:
import redis
import numpy as np
import pandas as pd
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.index_definition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from sentence_transformers import SentenceTransformer

# 1️⃣ Redis 연결
r = redis.Redis(host="localhost", port=6379, decode_responses=False)

# 2️⃣ 임베딩 모델 로드
print("🚀 ko-sroberta-multitask 모델 로드 중...")
model = SentenceTransformer("jhgan/ko-sroberta-multitask")
print("✅ 모델 로드 완료")

# 3️⃣ 임베딩 함수
def get_embedding(text: str):
    emb = model.encode(text, normalize_embeddings=True)
    return emb.tolist()

# 4️⃣ Parquet 로드
df = pd.read_parquet("qa_dataset.parquet", engine="fastparquet")

# 5️⃣ Redis 인덱스 설정
index_name = "qa_index"
try:
    r.ft(index_name).dropindex(delete_documents=True)
except Exception as e:
    print("⚠️ 기존 인덱스 없음 또는 삭제 완료:", e)

dim = len(get_embedding("임베딩 차원 확인"))

r.ft(index_name).create_index(
    fields=[
        VectorField("embedding", "FLAT", {
            "TYPE": "FLOAT32",
            "DIM": dim,
            "DISTANCE_METRIC": "COSINE"
        }),
        TextField("answer")
    ],
    definition=IndexDefinition(prefix=["doc:"], index_type=IndexType.HASH)
)

# 6️⃣ Redis에 데이터 적재
for i, row in df.iterrows():
    emb = np.array(get_embedding(row["question"]), dtype=np.float32).tobytes()
    r.hset(f"doc:{i}", mapping={
        "embedding": emb,
        "answer": row["answer"]
    })

print("✅ Redis에 ko-sroberta 임베딩 저장 완료")

# 7️⃣ 유사도 검색 함수 추가 (RedisInsight 쿼리 자동 실행용)
def search_similar(query_text: str, k: int = 5):
    """
    주어진 query_text와 가장 유사한 answer를 Redis에서 검색
    """
    # 쿼리 임베딩 생성
    q_emb = np.array(model.encode(query_text, normalize_embeddings=True), dtype=np.float32).tobytes()

    # FT.SEARCH 쿼리 구성
    q = (
        Query(f"*=>[KNN {k} @embedding $vec AS score]")
        .return_fields("answer", "score")
        .sort_by("score")
        .dialect(2)
    )

    # Redis 검색 실행
    res = r.ft(index_name).search(q, query_params={"vec": q_emb})

    print(f"\n🔍 '{query_text}' 과(와) 유사한 상위 {k}개 결과:")
    for doc in res.docs:
        print(f"- {doc.answer} (유사도 점수: {doc.score})")

🚀 ko-sroberta-multitask 모델 로드 중...
✅ 모델 로드 완료
✅ Redis에 ko-sroberta 임베딩 저장 완료


In [80]:
# 사용자 질의 → 임베딩 자동 생성
# LLM 입력 전에 사용자의 query를 임베딩
def embed_query(query: str):
    return np.array(get_embedding(query), dtype=np.float32).tobytes()

In [87]:
# Redis에서 유사 검색 → 답변 반환
# Query 임베딩과 저장된 embedding 비교 → 가장 유사한 answer 반환

from redis.commands.search.query import Query

def search_answer(query: str):
    q_emb = embed_query(query)

    q = Query('*=>[KNN 1 @embedding $vec]') \
        .return_fields("answer", "__score__") \
        .dialect(2)

    res = r.ft(index_name).search(q, query_params={"vec": q_emb})
    if res.docs:
        return res.docs[0]["answer"]
    else:
        return "❌ 답변을 찾을 수 없습니다."

# 테스트
query = "관세법 제97조 재수출면세 대상 및 재수출 시기는?"
print("🔍 사용자 질의:", query)
print("📌 검색된 답변:", search_answer(query))

# #작동 요약:

# embed_query(query) → 사용자 입력 문장을 768차원 float32 벡터로 변환

# Redis 인덱스(qa_index)의 embedding 필드와 코사인 거리(COSINE)로 비교

# 가장 가까운 벡터(KNN 1)를 찾아 해당 answer 필드 반환

🔍 사용자 질의: 관세법 제97조 재수출면세 대상 및 재수출 시기는?
📌 검색된 답변: 「대외무역법」 제2조제1호에서는 ‘물품’과 ‘전자적 형태의 무체물’을 구분 하여 규정하고 있으며, 「대외무역법」상 ‘무역’은 물품뿐만 아니라 용역, 전자적 형태의 무체물의 수출과 수입을 포함하고 있습니다. 그러나「관세법」에 따른 ‘수입’이란 외국물품을 우리나라에 반입(보세구역을 경유하는 것은 보세구역으로부터 반입하는 것)하거나 우리나라에서 소비 또는 사용하는 것(우리나라의 운송수단 안에서의 소비 또는 사용을 포함 하며, 법 제239조에 따른 수입으로 보지 아니하는 소비 또는 사용은 제 외)을 말합니다. 즉, 「관세법」에서는 공간의 일부를 차지하고 유형적 존재를 가지는 물건을 일반적으로 의미하는 ‘유체물’의 국가 간 반출입을 수입신고 등을 해야 하는 대상으로 정하고 있으므로, 소프트웨어 등 대외무역법령에 따른 ‘전자적 형태의 무체물’은「관세법」에 따른 수입신고의 대상이 아닙니다. \ 관련법령 ● 관세법 제2조 및 제241조 ● 대외무역법 제2조 016|관세행정 민원상담 사례집



In [85]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 1️⃣ 임베딩 모델 로드
print("🚀 ko-sroberta-multitask 모델 로드 중...")
model = SentenceTransformer("jhgan/ko-sroberta-multitask")
print("✅ 모델 로드 완료")

# 2️⃣ Parquet 파일 로드
df = pd.read_parquet("qa_dataset.parquet", engine="fastparquet")

# 3️⃣ 전체 질문 임베딩 생성
print("📊 전체 질문 임베딩 계산 중...")
df["embedding"] = df["question"].apply(lambda x: model.encode(x, normalize_embeddings=True))
print("✅ 전체 임베딩 완료")

# 4️⃣ 유사도 검색 함수
def search_similar(query_text: str, k: int = 5):
    """
    입력된 query_text와 가장 유사한 질문(answer) Top-K를 출력
    """
    query_vec = model.encode(query_text, normalize_embeddings=True).reshape(1, -1)
    all_vecs = np.vstack(df["embedding"].values)

    # 코사인 유사도 계산
    sims = cosine_similarity(query_vec, all_vecs)[0]

    # 상위 K개 인덱스 정렬
    top_k_idx = np.argsort(sims)[::-1][:k]
    
    print(f"\n🔍 '{query_text}' 과(와) 가장 유사한 {k}개 문장:")
    for i, idx in enumerate(top_k_idx, start=1):
        print(f"{i}. {df.iloc[idx]['question']}")
        print(f"   💬 Answer: {df.iloc[idx]['answer']}")
        print(f"   🎯 Similarity: {sims[idx]:.4f}\n")

# 5️⃣ 테스트
if __name__ == "__main__":
    test_query = "여행자 휴대품 면세 범위가 어떻게 돼?"
    search_similar(test_query, k=5)


🚀 ko-sroberta-multitask 모델 로드 중...
✅ 모델 로드 완료
📊 전체 질문 임베딩 계산 중...
✅ 전체 임베딩 완료

🔍 '여행자 휴대품 면세 범위가 어떻게 돼?' 과(와) 가장 유사한 5개 문장:
1. 여행자휴대품의 면세범위는 어떻게 되는지요?
   💬 Answer: 여행자가 해외(국내·외 면세점 포함)에서 취득(구입·선물 포함)한 물품을 국내로 휴대하여 반입할 때 해외에서 취득한 물품 가격의 총합이 미화 800달러 이하인 경우 관세가 면제됩니다. 술, 향수, 담배는 기본 면세범위를 적용하지 않으며 별도로 면세범위를 적용하고 있습니다. 구분 면세물품 면세범위 비 고 자가사용, 선물용 등에 한함 해외 취득물품 합계액 상용물품, 수리용품·견본품 등 (국내면세점 구매물품 포함) 회사용품 적용배제 기본 US$800 (US$800 공제 불가) 면세 전체 취득가격 10만원 이내, 품목별 농림축수산물 및 한약재 및 전체 중량 제한 별도로 있음 술 2병 합산 2ℓ이하로서 US$400이하 별도 담배(궐련인 경우) 200개비 가격제한 없음(단, 한 종류만) 면세 향수 100㎖ 용량제한만 있음(수량무관) 관련법령 ● 관세법 시행규칙 제48조 PART 1 통 관|193
통
관
구분 | 면세물품 | 면세범위 | 비 고
 | 해외 취득물품 합계액
(국내면세점 구매물품 포함) | US$800 | 
 | 농림축수산물 및 한약재 |  | 
 | 술 | 2병 | 
 | 담배(궐련인 경우) | 200개비 | 
 | 향수 | 100㎖ |
   🎯 Similarity: 0.9249

2. 차(茶) 면세통관 범위는 어떻게 되나요?
   💬 Answer: 수입물품이 개인 자가사용이고 물품가격 미화 150불 이하(미국에서 수입된물품은 200불 이하)인 경우, 특송업체가 세관장에게 통관목록을 제출함으로써 수입신고가 생략되고 관세등 제세 납부없이 통관(목록통관) 되나, 문의하신 차(茶)와 같은 식품류는 「특송물품 수입통관 사무처리에 관한 고시」 별표1 ‘목록통관 배제대상